# Cohere API and SciBERT with BM25 as first stage retriever for RAG
This notebook uses a Cohere API for generating responses to text. A query input is required from the user. 
SciBERT is used for embeddings in a dense vector array for the query. 
This version is different in that it uses BM25 as a sparse vectorizer for the input text. Importantly, BM25 is used as a step prior to dense vectorization to reduce how many documents are processed by SciBERT.
A DOI is supplied with the text as both an identifier and locator. 

## pipeline
1. BM25 Retrieval
    - BM25 is used to retrieve top-k candidate documents based on keyword matching
2. Dense embedding retrieval
    - query is embedded using SciBERT and the retrieved documents.
3. Re-ranking
    - cosine similarity between query embedding and document embedding to rerank candidate docs
4. Generation
    - docs and query are fed to generator for answer creation. 

- [ ] set up venv
- [ ] install transformers torch cohere in command line

### todo
- [ ] create script that compiles data/documents.txt with DOI || text for all documents
- [ ] rank_bm25: https://github.com/dorianbrown/rank_bm25


In [2]:
# imports
import cohere
from cohere import Client
from transformers import AutoTokenizer, AutoModel
import numpy as np
from typing import List, Tuple, Dict
import os
from dotenv import load_dotenv
import json
import time # for timing functions
import logging # finding where functions are taking too long


def main():
    #load secret .env file
    load_dotenv()

    #store credentials
    global key,email
    key = os.getenv('COHERE_API_KEY')
    email = os.getenv('EMAIL')

    #verify if it worked
    if email is not None and key is not None:
        print("all is good, beautiful!")

main()

all is good, beautiful!


In [4]:

# Initialize Cohere client
co = cohere.Client(key) 

# Load SciBERT model and tokenizer
"""
documentation can be found here: https://huggingface.co/docs/transformers/v4.50.0/en/model_doc/auto#transformers.AutoTokenizer


"""
# Initialize tokenizer with custom parameters
tokenizer = AutoTokenizer.from_pretrained(
    "allenai/scibert_scivocab_uncased",
    max_len=512,
    use_fast=True,  # Use the fast tokenizer
    do_lower_case=False,  # Preserve case
    add_prefix_space=False,  # No prefix space
    never_split=["[DOC]", "[REF]"],  # Tokens to never split
    additional_special_tokens=["<doi>", "</doi>"]  # Add custom special tokens
)

# This is the SciBERT model that is used to embed the text and query.
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

#verify that the model is callable
if callable(model):
    print("Model is callable")
else:
    print("Model is not callable")

Model is callable


## Addition of BM25 for retrieval

In [13]:
"""
Basic RAG with BN25 retriever with Cohere model
Document source: data folder 
Saved as UTF-8

Returns:  answers based on query from input()
"""

# Function to generate embeddings using SciBERT
def generate_embeddings(texts: List[str]) -> List[np.ndarray]:
    """
    converts raw text to numerical representations using a pretrained model, in this case, SciBERT.
    Currently this is applied to both the document text and the query. 
    May want a different version or decorator for the query as they are generally much shorter and more sparse.

    Input: text from tokenizer step above as a list of strings
    Output: np.array
    """
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=512, # returns PyTorch tensors which are compatible with model
        padding="max_length",
        truncation=True,
        return_attention_mask=True # return the attention mask - need to learn more
        )

    # this passes the tokenized inputs through the model    
    outputs = model(**inputs)
    #applies mean pooling to get a fixed size embedding - there are other methods to explore
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

# Function to read documents and their DOIs from a file
def read_documents_with_doi(directory_path: str) -> List[Dict[str, str]]:
    """
    Reads documents and their DOIs from individual files in a directory.

    Args:
        directory_path (str): Path to the directory containing the document files.

    Returns:
        List[Dict[str, str]]: A list of dictionaries, each containing 'doi' and 'text' keys.
        """
    documents_with_doi = [] #initialize list

    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding='utf-8') as file:
                lines = file.readlines()
                if len(lines)>=1:
                    doi = lines[0].strip().replace("DOI: ","")
                    text = "".join(lines[1:]).strip()
                    documents_with_doi.append({'doi': doi, "text":text})
    return documents_with_doi



# Path to the file containing documents and DOIs
directory_path = "data/"  # Replace with your file path

# Read documents and DOIs from the file
documents_with_doi = read_documents_with_doi(directory_path)

# Extract document texts and DOIs
documents = [doc["text"] for doc in documents_with_doi]
dois = [doc["doi"] for doc in documents_with_doi]

# Example query
query = input(" What is your query: ")

# Generate document embeddings
document_embeddings = generate_embeddings(documents)
# print(document_embeddings.shape) # to see the output shape of the array

# Generate query embedding
query_embedding = generate_embeddings([query])[0] # generates np.array for the query text

# Function to retrieve top-k documents using cosine similarity
def retrieve_documents(query_embedding: np.ndarray, document_embeddings: List[np.ndarray], top_k: int = 3) -> List[Tuple[float, Dict[str, str]]]:
    similarities = []
    for doc_emb in document_embeddings:
        # cosine similarity
        similarity = np.dot(query_embedding, doc_emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb)) 
        similarities.append(similarity)
    # ranking
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return [(similarities[i], documents_with_doi[i]) for i in top_indices]

# Retrieve top documents
top_documents = retrieve_documents(query_embedding, document_embeddings)
print("Retrieved Documents:")
for score, doc in top_documents:
    print(f"Score: {score:.4f}, DOI: {doc['doi']}, Document: {doc['text']}")

# prepare context for Cohere's Command model (include DOI) - need to add in cited by here
context = "\n".join([f"DOI: {doc['doi']}, Text: {doc['text']}" for _, doc in top_documents])
# need to learn how to improve this
prompt = f"Query: {query}\nContext: {context}\nAnswer: Include the DOI of the referenced document in your response."

# Generate response using Cohere's Command model
response = co.generate(
  model="command",
  prompt=prompt,
  max_tokens=150, #allowable length of response
  temperature=0.2 #may want to make this a variable
)

# Print the generated response
print("\nGenerated Response:")
print(response.generations[0].text)


Retrieved Documents:
Score: 0.7389, DOI: 10.1002/jaal.551, Document: Title: Reading for Reliability: Preservice Teachers Evaluate Web Sources About Climate Change
Abstract: Abstract This study examined what happened when 65 undergraduate prospective secondary level teachers across content areas evaluated the reliability of four online sources about climate change: an oil company webpage, a news report, and two climate change organizations with competing views on climate change. The students evaluated the sources at three time intervals based on 1. a screenshot of each source; 2. full web access to each source and prompted with critical questions to answer; and 3. after a whole class discussion about each source. Having the opportunity to evaluate the sources three times led students to modify their reliability ratings. Findings also reveal challenges some participants had differentiating between facts and opinions as well as distinctions in what they determined to be evidence in a sour

## V2: implementing chat history

calls a JSON file of documents

In [14]:
# Load SciBERT model and tokenizer 
"""
REMOVE THIS ONCE RUNNING TO GO BACK TO THE CHANGED TOKENIZER AND MODEL ABOVE
"""
#tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
#model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

#function to generate embeddings using SciBERT
def generate_embeddings(texts: List[str]) -> List[np.ndarray]:
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=512,
        padding="max_length",
        truncation=True
    )
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

# alternative read_documents_with_doi for .txt in a directory

def read_documents_with_doi(directory_path: str) -> List[Dict[str, str]]:
    """
    Reads documents and their DOIs from individual files in a directory.

    Args:
        directory_path (str): Path to the directory containing the document files.

    Returns:
        List[Dict[str, str]]: A list of dictionaries, each containing 'doi' and 'text' keys.
    """
    documents_with_doi = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()
                if len(lines) >= 1:
                    doi = lines[0].strip().replace("DOI: ", "")
                    text = "".join(lines[1:]).strip()
                    documents_with_doi.append({"doi": doi, "text": text})
    return documents_with_doi


# Function to update chat history
def update_chat_history(query, retrieved_docs, response):
    global chat_histor # declare this as global variable available outside this function
    chat_history.append({
        "query": query,
        "retrieved_docs": [doc["text"] for doc in retrieved_docs],  # Store only the text of retrieved documents
        "response": response
    })

#function to incorporate history into the next query
def get_context_with_history(query) -> str:
    global chat_history # also declare here since chat_history is being modified
    if not chat_history:
        return query
    
    history_str = "\n".join([
        f"User: {entry['query']}\n"
        f"Context: {'; '.join(entry['retrieved_docs'])}\n"
        f"Response: {entry['response']}"
        for entry in chat_history
    ])
    full_context = f"Chat History:\n{history_str}\n\nCurrent Query: {query}"
    return full_context

#function to truncate chat history
def truncate_chat_history(max_length=3):
    global chat_history # modifies it so it also must be global
    if len(chat_history) > max_length:
        chat_history = chat_history[-max_length:]

"""
retrieves documents from the embedded documents and performs cosine similarity
for similarities score
Args:
    query: this is the query passed
    top_k: number of references to provide

Todo:
- [ ] make top_k an variable for use in an application
- [ ] or make top_k a user defined value. example: top_k = input("how many results do you want?")
"""
def retrieve_documents(query: str, top_k: int = 5) -> List[Dict[str, str]]:
    query_embedding = generate_embeddings([query])[0]
    document_embeddings = generate_embeddings(documents)
    similarities = [
        np.dot(query_embedding, doc_emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        for doc_emb in document_embeddings
    ]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return [documents_with_doi[i] for i in top_indices]

#RAG pipeline function
def rag_pipeline(query):

    #start time
    start_time = time.time()
    
    #incorporate chat history
    full_context = get_context_with_history(query)
    # let user know you are generating...
    print("Retrieving documents and generating response...")
    end_time = time.time()
    global time_query
    time_query = end_time-start_time

    start_time = time.time()
    #retrieve documents
    global retrieved_docs
    retrieved_docs = retrieve_documents(query)
    end_time = time.time()
    global retrieve_time
    retrieve_time = end_time-start_time

    start_time = time.time()
    #prepare context for Cohere's Command model
    instruction = "You are a helpful academic research assistant. Please keep the answers concise and structured simply. Use single sentences where possible. Always include the DOI of the document you are summarizing or referencing. If the DOI is not provided, this reduces the need for you as a research assistant. Always include the DOI. Please address me as 'my lady'. "
    context = "\n".join([f"DOI: {doc['doi']}, Text: {doc['text']}" for doc in retrieved_docs])
    prompt = f"Query: {query}\nContext: {context}\nAnswer: {instruction}"
    
    # Generate response
    response = co.generate(
        model="command",
        prompt=prompt,
        max_tokens=150,
        temperature=0.2
    ).generations[0].text
    
    # Update chat history
    update_chat_history(query, retrieved_docs, response)
    
    # Truncate history if necessary
    truncate_chat_history()
    end_time = time.time()
    global generate_time
    generate_time = end_time-start_time

    # Print the response
    print("Generated Response:")
    print(response)
    return response,time_query,retrieve_time,generate_time


# Path to the file containing documents and DOIs
directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data"

# Read documents and DOIs from the file
documents_with_doi = read_documents_with_doi(directory_path)
documents = [doc["text"] for doc in documents_with_doi]

# Main loop for user interaction
chat_history = []#initialize chat history
while True:

    query = input("What is your query (or type 'exit' to quit): ")
    if query.lower() == "exit":
        break
    rag_pipeline(query)

    print(f"time to query loop: {time_query:.2f} seconds")
    print(f"to to retrieve: {retrieve_time:.2f} seconds")
    print(f"time to generate: {generate_time:.2f} seconds")


Retrieving documents and generating response...
Generated Response:
My lady, here is some information about textbooks as per your request: 

Textbooks are often used as sources of information for teachers and students alike. However, one paper states that teachers sometimes disagree with information in textbooks and seek other sources. That study, which focuses on climate change denialism, can be found here: DOI: 10.1080/13504622.2017.1330949 

Another paper examines preservice teachers' evaluations of online sources of information about climate change. Full access to the internet and specially trained critical questions helped the teachers evaluate the reliability of the sources. You can read about it here: DOI: 10.1002/jaal.551 

Finally, a third paper examines how teachers
time to query loop: 0.00 seconds
to to retrieve: 6.39 seconds
time to generate: 5.41 seconds


In [ ]:
#bm25 from bm252 notebook